In [1]:
pip install beautifulsoup4 requests pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import bs4
import pandas as pd
import requests

In [9]:
from time import sleep
from random import randint

In [10]:
#Set the pages & years 
pages = [str(i) for i in range(1,20)]
years_url = [str(i) for i in range(1990,2019)]

In [11]:
#Get page contents function
def get_page_contents(url):
    page = requests.get(url)
    return bs4.BeautifulSoup(page.text, "html.parser")

In [12]:
#Get movie titles and release years
def get_title_year(soup): 
    movies = soup.findAll('h3', class_='lister-item-header')
    titles = [movie.find('a').text for movie in movies]
    release = [movie.find('span', class_='lister-item-year text-muted unbold').text[-5:-1] for movie in movies]
   # bad_chars = ['(I) ']
    #final1=[]
    #for i in bad_chars:
     #   for j in release:
      #      j = j.replace(i,'').replace("(","").replace(")","")
       #     final1.append(j)
    DF=pd.DataFrame()
    DF['MovieTitle']=titles
    DF['ReleaseYear']=release
    return DF

In [13]:
#Function to get the IMDB rating
def get_rating(soup):
    movies1=soup.findAll('div', class_="ratings-bar")
    rating=[]
    for movie in movies1:
        rating.append(float(movie.find('div', 'inline-block ratings-imdb-rating')['data-value']))
    DF=pd.DataFrame()
    DF['Rating']=rating
    return DF

In [14]:
#Function to get the votes & earnings
def get_votes_earnings(soup):
    movies2=soup.findAll('p', class_='sort-num_votes-visible')
    num=[]
    votes=[]
    earnings=[]
    votes_available=1
    earnings_available=1
    i=0
    for movie in movies2:
        num.append(movie.findAll('span' , attrs = {'name' : 'nv'}))
        try:
            v=str(num[i][0])
        except:
            votes_available=0
        try:
            e=str(num[i][1])
        except:
            earnings_available=0
        if votes_available!=0: votes.append(v.split('"')[1])
        else: votes.append(None)
        if  earnings_available!=0: earnings.append(e.split('"')[1])
        else: earnings.append(None)
        i=i+1  
    DF=pd.DataFrame()
    DF['Votes']=votes
    DF['GrossEarnings']=earnings
    return DF

In [15]:
#Function to get the genre,runtime & certificate
def get_genre_runtime(soup):
    movies3=soup.findAll('p', class_='text-muted')
    genre=[]
    runtime=[]
    certificate=[]
    i=0
    a = [i for i in range(100)]
    for i in a:
        if i%2==0:
            a=(movies3[i].find('span' , class_='genre').text).lstrip()
            genre.append(str(a).strip())
            runtime.append(movies3[i].find('span' , class_='runtime').text)
            certificate.append(movies3[i].find('span' , class_='certificate').text)
    DF=pd.DataFrame()
    DF['Genre']=genre
    DF['RunTime']=runtime
    DF['Certificate']=certificate
    return DF

In [16]:
#Function to get the meta score of the movies
def get_metascore(soup):
    movies4=soup.findAll('div',class_='inline-block ratings-metascore')
    metascore=[]
    metascore_param=[]
    for movie in movies4:
        try:
            metascore.append(movie.find('span',class_='metascore favorable').text)
            metascore_param.append('Favorable')
        except:
            try:
                metascore.append(movie.find('span',class_='metascore mixed').text)
                metascore_param.append('Mixed')
            except:
                metascore.append(movie.find('span',class_='metascore unfavorable').text)
                metascore_param.append('Unfavorable')
    DF=pd.DataFrame()
    DF['Metascore_Value']=metascore
    DF['Metascore_Key']=metascore_param
    return DF

In [17]:
#Pull the data - web scraping
DF_Final=pd.DataFrame()
DF_new=pd.DataFrame()
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        url='http://www.imdb.com/search/title?release_date=' + year_url + '&sort=num_votes,desc&page=' + page
        soup = get_page_contents(url)
        DF1=get_title_year(soup)
        DF2=get_genre_runtime(soup)
        DF3=get_rating(soup)
        DF4=get_votes_earnings(soup)
        DF5=get_metascore(soup)
        DF_new=pd.concat([DF1,DF2,DF3,DF4,DF5], axis=1, ignore_index=True)
        DF_Final=DF_Final.append(DF_new) 
        

In [18]:
#Check the data frame dimensions
DF_Final.shape

(27550, 10)

In [19]:
#Renaming the columns
DF_Final.columns = ['Title','ReleaseYear','Genre','Duration','Certificate','IMDB_Rating','Votes','GrossIncome','Metascore_Value','Metascore_Key']

In [20]:
#Checking the data types
DF_Final.dtypes

MovieTitle          object
ReleaseYear         object
Genre               object
Duration            object
Certificate         object
IMDB_Rating        float64
Votes               object
GrossIncome         object
Metascore_Value     object
Metascore_Key       object
dtype: object

In [87]:
#DF_Final=DF_Final.reset_index(drop=True)

In [21]:
#check some records
DF_Final.head()

,MovieTitle,ReleaseYear,Genre,Duration,Certificate,IMDB_Rating,Votes,GrossIncome,Metascore_Value,Metascore_Key
0,Goodfellas,1990,"Biography, Crime, Drama",146 min,R,8.7,961282,"46,836,394",90,Favorable
1,Home Alone,1990,"Comedy, Family",103 min,PG,7.6,442270,"285,761,243",63,Favorable
2,Edward Scissorhands,1990,"Drama, Fantasy, Romance",105 min,PG-13,7.9,429128,"56,362,352",74,Favorable
3,Back to the Future Part III,1990,"Adventure, Comedy, Sci-Fi",118 min,PG,7.4,380294,"87,727,583",55,Mixed
4,The Godfather: Part III,1990,"Crime, Drama",162 min,R,7.6,339644,"66,666,062",60,Mixed


In [22]:
#Filter the movie data - remove the drams's data
DF_IMDB=pd.DataFrame()
DF_IMDB=DF_Final[DF_Final['Certificate'].isin(['R','G','PG','PG-13','NC-17']) ]

In [28]:
#Check the final data frame dimensions
DF_IMDB.shape

(25308, 10)

In [24]:
#check some records
DF_IMDB.head()

,MovieTitle,ReleaseYear,Genre,Duration,Certificate,IMDB_Rating,Votes,GrossIncome,Metascore_Value,Metascore_Key
0,Goodfellas,1990,"Biography, Crime, Drama",146 min,R,8.7,961282,"46,836,394",90,Favorable
1,Home Alone,1990,"Comedy, Family",103 min,PG,7.6,442270,"285,761,243",63,Favorable
2,Edward Scissorhands,1990,"Drama, Fantasy, Romance",105 min,PG-13,7.9,429128,"56,362,352",74,Favorable
3,Back to the Future Part III,1990,"Adventure, Comedy, Sci-Fi",118 min,PG,7.4,380294,"87,727,583",55,Mixed
4,The Godfather: Part III,1990,"Crime, Drama",162 min,R,7.6,339644,"66,666,062",60,Mixed


In [25]:
#Drop duplicates if any
DF_Updated=DF_IMDB.drop_duplicates(subset=None, keep='first', inplace=False)
DF_Updated.shape

In [29]:
#Write the data into a csv file
DF_Updated.to_csv('IMDB_data_Updated.csv', index = False)